**Task 1**

Task 1(a):

In [ ]:
from IPython.core.display import display, HTML
import pandas as pd


!wget https://norvig.com/ngrams/count_1w.txt #unigram data
!wget https://norvig.com/ngrams/count_2w.txt #bigram data


filePath1 = "/content/count_1w.txt"
filePath2 = "/content/count_2w.txt"


unigrams_df = pd.read_csv(filePath1,sep='\t',header=None, names=['unigram','count'])
bigrams_df = pd.read_csv(filePath2,sep='\t',header=None, names=['bigram','count'])


print(unigrams_df.shape, bigrams_df.shape)
print(f'Number of unigrams: {unigrams_df.shape[0]} Number of Bigrams: {bigrams_df.shape[0]}')  # df.size VS df.shape


display(unigrams_df.head(100),bigrams_df.head(100))




--2025-10-30 20:37:37--  https://norvig.com/ngrams/count_1w.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4956241 (4.7M) [text/plain]
Saving to: ‘count_1w.txt’

count_1w.txt        100%[===================>]   4.73M  6.00MB/s    in 0.8s    

2025-10-30 20:37:38 (6.00 MB/s) - ‘count_1w.txt’ saved [4956241/4956241]

--2025-10-30 20:37:38--  https://norvig.com/ngrams/count_2w.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5566017 (5.3M) [text/plain]
Saving to: ‘count_2w.txt’

count_2w.txt        100%[===================>]   5.31M  6.65MB/s    in 0.8s    

2025-10-30 20:37:39 (6.65 MB/s) - ‘count_2w.txt’ saved [5566017/5566017]

(333333, 2) (286358, 2)
Number of unigrams: 333333 Number of Bigrams: 286358


,unigram,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698
...,...,...
95,like,520585287
96,service,519537222
97,x,508609523
98,than,502609275


,bigram,count
0,0Uplink verified,523545
1,0km to,116103
2,1000s of,939476
3,100s of,539389
4,100th anniversary,158621
...,...,...
95,24th of,327460
96,25th anniversary,261023
97,25th of,397735
98,26th of,271707


Task 1(b):

In [ ]:
def probability(sentence):
  words = sentence.split()
  print("\nsentence ",words)
  #print("Probability obtained using unigrams:")
  prob1=1
  total_unigrams = unigrams_df['count'].sum()
  unigram = dict(zip(unigrams_df['unigram'], unigrams_df['count']))
  print(unigram)
  if words[0] not in unigram:
    prob1*=0
    #print("probability of sentence obtained using unigram is:",prob1)
  else:
    for i in range(len(words)):
      prob1 *= unigram[words[i]]/total_unigrams
    #print("probability of sentence obtained using unigram is:",prob1)

  #print("probability of bgram:-")
  total_bigrams = bigrams_df['count'].sum()
  bigram = dict(zip(bigrams_df['bigram'], bigrams_df['count']))
  #print(bigram)
  prob2=1
  for i in range(1,len(words)):
    prev=words[i-1]
    curr=words[i]
    w1=prev+" "+curr
    if prev+" "+curr not in bigram:
      prob2*=0
      w1=0
      if prev not in unigram:
        w3=0
      else:
       w3=unigram[words[i-1]]
      print("\nBigram count of ",w1,"is : 0")
      print("\nunigram count of ",prev,"is : ",w3)
      print("\nbigram probability of ",w1," is :",prob2)
    else:
      print(w1)
      w2=bigram[words[i-1]+" "+words[i]]
      w3=unigram[words[i-1]]
      print("\nBigram count of ",w1,"is : ",w2)
      print("\nunigram count of ",prev,"is : ",w3)
      k=w2/w3
      print("\nbigram probability of ",w1," is :",k)
      prob2 *= k
  print(f"\nsentence Probability of {sentence} is :",prob2)
  return prob2

probA = probability("i love you")
brobB = probability('i hate you')
print(f'\nprobability("i love you")>probability("i hate you") is: {probA>brobB}\n')
probability("OOV1 OOV2") # Test for OOV (Out Of Vocabulary) N-grams


sentence  ['i', 'love', 'you']
i love

Bigram count of  i love is :  3979312

unigram count of  i is :  3086225277

bigram probability of  i love  is : 0.001289378332053626
love you

Bigram count of  love you is :  5428714

unigram count of  love is :  201063526

bigram probability of  love you  is : 0.02699999402178991

sentence Probability of i love you is : 3.481320725727335e-05

sentence  ['i', 'hate', 'you']
i hate

Bigram count of  i hate is :  876611

unigram count of  i is :  3086225277

bigram probability of  i hate  is : 0.0002840398614232463
hate you

Bigram count of  hate you is :  504048

unigram count of  hate is :  21274675

bigram probability of  hate you  is : 0.023692394830943365

sentence Probability of i hate you is : 6.7295845445659906e-06

probability("i love you")>probability("i hate you") is: True


sentence  ['OOV1', 'OOV2']

Bigram count of  0 is : 0

unigram count of  OOV1 is :  0

bigram probability of  0  is : 0

sentence Probability of OOV1 OOV2 is : 0


0

Task 1(c) :add-one smoothing

In [ ]:
def probability(sentence):
  words = sentence.split()
  print("\nsentence ",words)
  #print("Probability obtained using unigrams:")
  prob1=1
  total_unigrams = unigrams_df['count'].sum()
  unigram = dict(zip(unigrams_df['unigram'], unigrams_df['count']))
  #print(unigram)
  if words[0] not in unigram:
    prob1*=0
    #print("probability of sentence obtained using unigram is:",prob1)
  else:
    for i in range(len(words)):
      prob1 *= unigram[words[i]]/total_unigrams
    #print("probability of sentence obtained using unigram is:",prob1)

  #print("probability of bgram:-")
  total_bigrams = bigrams_df['count'].sum()
  bigram = dict(zip(bigrams_df['bigram'], bigrams_df['count']))
  #print(bigram)
  prob2=1
  for i in range(1,len(words)):
    prev=words[i-1]
    curr=words[i]
    v=len(bigram)
    w1=prev+" "+curr
    if prev+" "+curr not in bigram:
      prob2*=0
      w2=0
      if prev not in unigram:
        w3=0
      else:
       w3=unigram[words[i-1]]
      print("\nBigram count of ",w2,"is : 0")
      print("\nunigram count of ",prev,"is : ",w3)
      k=(w2+1)/(w3+v)
      print("\nbigram probability of ",w1," is :",k)
      prob2=k
    else:
      print(w1)
      w2=bigram[words[i-1]+" "+words[i]]
      w3=unigram[words[i-1]]
      print("\nBigram count of ",w1,"is : ",w2)
      print("\nunigram count of ",prev,"is : ",w3)
      k=(w2+1)/(w3+v)
      print("\nbigram probability of ",w1," is :",k)
      prob2 *= k
  print(f"\nsentence Probability of {sentence} is :",prob2)
  return prob2

probA = probability("i love you")
brobB = probability('i hate you')
print(f'\nprobability("i love you")>probability("i hate you") is: {probA>brobB}\n')
probability("OOV1 OOV2") # Test for OOV (Out Of Vocabulary) N-grams


sentence  ['i', 'love', 'you']
i love

Bigram count of  i love is :  3979312

unigram count of  i is :  3086225277

bigram probability of  i love  is : 0.0012892590310938517
love you

Bigram count of  love you is :  5428714

unigram count of  love is :  201063526

bigram probability of  love you  is : 0.026961599838815898

sentence Probability of i love you is : 3.4760486084931935e-05

sentence  ['i', 'hate', 'you']
i hate

Bigram count of  i hate is :  876611

unigram count of  i is :  3086225277

bigram probability of  i hate  is : 0.00028401383298203573
hate you

Bigram count of  hate you is :  504048

unigram count of  hate is :  21274675

bigram probability of  hate you  is : 0.023377776009155034

sentence Probability of i hate you is : 6.6396117709556e-06

probability("i love you")>probability("i hate you") is: True


sentence  ['OOV1', 'OOV2']

Bigram count of  0 is : 0

unigram count of  OOV1 is :  0

bigram probability of  OOV1 OOV2  is : 3.4921322260946087e-06

sentence Prob

3.4921322260946087e-06

Task 2:Shannon visualisation

In [ ]:
import random
import pandas as pd
from IPython.core.display import display, HTML

!wget -q https://norvig.com/ngrams/count_2w.txt
filePath2 = "/content/count_2w.txt"
bigrams_df = pd.read_csv(filePath2, sep='\t', header=None, names=['bigram', 'count'])

def ShannonVisualization(seed="<S>"):
    sentence = [seed]
    current_seed = seed

    while True:
        mask = bigrams_df['bigram'].str.startswith(current_seed + " ")
        candidate_bigrams = bigrams_df[mask]

        if candidate_bigrams.empty:
            print(f"\nNo more bigrams starting with '{current_seed}'. Ending generation.")
            break

        total_count = candidate_bigrams['count'].sum()
        candidate_bigrams = candidate_bigrams.assign(
            prob=candidate_bigrams['count'] / total_count,
            cum_prob=(candidate_bigrams['count'] / total_count).cumsum()
        )

        r = random.random()
        chosen_row = candidate_bigrams[candidate_bigrams['cum_prob'] >= r].iloc[0]
        chosen_bigram = chosen_row['bigram']
        next_word = chosen_bigram.split()[1]

        print(f"Seed='{current_seed}', Bigram='{chosen_bigram}', Next='{next_word}'")
        sentence.append(next_word)
        current_seed = next_word
        if current_seed == "</S>":
            print("\nEnd of sentence reached (</S>)")
            break
        if current_seed == "and":
            print("\nStopping early after 'and' (test stop).")
            break


    result = " ".join(sentence)
    print(f"\nGenerated Sentence:\n{result}\n")
    return result
generated = ShannonVisualization("<S>")


Seed='<S>', Bigram='<S> begins', Next='begins'
Seed='begins', Bigram='begins with', Next='with'
Seed='with', Bigram='with a', Next='a'
Seed='a', Bigram='a per', Next='per'
Seed='per', Bigram='per person', Next='person'
Seed='person', Bigram='person would', Next='would'
Seed='would', Bigram='would never', Next='never'
Seed='never', Bigram='never see', Next='see'
Seed='see', Bigram='see other', Next='other'
Seed='other', Bigram='other than', Next='than'
Seed='than', Bigram='than for', Next='for'
Seed='for', Bigram='for different', Next='different'
Seed='different', Bigram='different type', Next='type'
Seed='type', Bigram='type of', Next='of'
Seed='of', Bigram='of total', Next='total'
Seed='total', Bigram='total loss', Next='loss'
Seed='loss', Bigram='loss of', Next='of'
Seed='of', Bigram='of the', Next='the'
Seed='the', Bigram='the exclusive', Next='exclusive'
Seed='exclusive', Bigram='exclusive and', Next='and'

Stopping early after 'and' (test stop).

Generated Sentence:
<S> begins wit